# PPO部分
- sample_action
  - 将reward缩小100倍，看看会不会减小梯度的剧烈变化
  - 不对sigma进行截断
  - 动作截断？
- act

In [ ]:
def act(self, obs, test_mode=False, exploration_scale=0.3):
    """
    改进版动作采样函数，针对稀疏奖励和连续动作空间优化
    参数：
        obs: 环境观测字典，必须包含'agent_obs'键
        test_mode: 测试模式开关，True时关闭探索噪声
        exploration_scale: 初始探索强度系数(0.1~0.5)
    返回：
        list: 2D动作 [action_dim1, action_dim2]
    """
    # 状态预处理
    state = torch.tensor(obs['agent_obs'], dtype=torch.float).to(self.device)
    state = state.unsqueeze(0)  # 添加batch维度 [1, ...]
    
    # 获取动作分布
    with torch.no_grad():
        mu, sigma, (self.actor_h, self.actor_c) = self.actor(
            self.update_history_sequence(state),
            self.actor_h, self.actor_c
        )
        
        # 约束标准差范围并检查异常
        sigma = torch.clamp(sigma, min=1e-3, max=1.0)
        if torch.isnan(mu).any() or torch.isnan(sigma).any():
            self.isnan = True
            print("警告: 动作分布异常，使用安全动作")
            mu = torch.zeros_like(mu)
            sigma = torch.ones_like(sigma) * 0.1
        else:
            self.isnan = False

    # 稀疏奖励环境专用探索策略
    if not test_mode:
        # 自适应探索噪声（训练步数越多噪声越小）
        decay_factor = max(0.1, 1 - self.sample_count / 2e6)  # 200万步衰减到10%
        noise = exploration_scale * decay_factor * torch.randn_like(mu) * sigma
        
        # 对关键维度加强探索（示例：第2维度）
        noise[:, 1] *= 1.5  # 第2维度的探索强度增加50%
        
        action = mu + noise
    else:
        action = mu  # 测试模式直接使用均值

    # 动作截断（根据环境物理限制）
    action = action.cpu().numpy()
    action[0, 0] = np.clip(action[0, 0], -100, 200)  # 第一维限制
    action[0, 1] = np.clip(action[0, 1], -30, 30)    # 第二维限制
    
    return action[0].tolist()

- update
  - 不对ratio作截断